<a href="https://colab.research.google.com/github/amhaiskar0921/BTTAIAmazonProject/blob/main/FinalRandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import pandas as pd
import numpy as np

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

In [44]:
df_merged = pd.read_csv('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/Datasets/balanced_preprocessed_merged_data.csv')
df_merged.head(20)

,product_id,product_locale,product_title,product_description,product_brand,esci_label,query,bert_input
0,B00BO9BGYU,es,light irridiance oxigenada 20vol 1000ml l negr...,NaN,light irridiance,I,yunsey tinte amoniaco,[CLS] yunsey tinte amoniaco [SEP] light irridi...
1,3836582864,es,peter lindbergh fashion photography 40th anniv...,NaN,taschen,I,libro channel decoracion,[CLS] libro channel decoracion [SEP] peter lin...
2,B085TCJ87R,us,lenovo thinkcentre m75s small form factor sff ...,NaN,lenovo,I,ryzen 11,[CLS] ryzen 11 [SEP] lenovo thinkcentr m75 sma...
3,B002T3RV5S,us,delta faucet foundations single function tub s...,NaN,delta faucet,I,tub faucet without shower head,[CLS] tub faucet without shower head [SEP] del...
4,B082KFTF41,us,kokodi lcd writing tablet 10 inch colorful tod...,features love 1 10 inch lcd colorful screen of...,kokodi,E,educational toys 3 year old,[CLS] educ toy 3 year old [SEP] kokodi lcd wri...
5,B01N0DZF25,jp,小学館の図鑑neo 新版 動物,ご注意 お使いの端末によっては 一部読みづらい場合がございます お手持ちの端末で立ち読みファ...,NaN,I,学校に行きたくない君へ,[CLS] 学校 に 行き たく ない 君 へ [SEP] 小学 館 の 図鑑 neo 新版...
6,B01N0DZF25,jp,小学館の図鑑neo 新版 動物,ご注意 お使いの端末によっては 一部読みづらい場合がございます お手持ちの端末で立ち読みファ...,NaN,I,子なし のリアル,[CLS] 子 なし の リアル [SEP] 小学 館 の 図鑑 neo 新版 動物 [SE...
7,B084GXV474,us,tracfone carrier locked samsung galaxy j7 crow...,NaN,tracfone,E,android phone,[CLS] android phone [SEP] tracfon carrier lock...
8,B084GXV474,us,tracfone carrier locked samsung galaxy j7 crow...,NaN,tracfone,E,used phones,[CLS] use phone [SEP] tracfon carrier lock sam...
9,B08467FBSR,us,hind 4pk active workout athletic shorts girls ...,hind 4 pack athletic shorts girls perfect pack...,hind,S,soccer shorts girls,[CLS] soccer short girl [SEP] hind 4pk activ w...


##Getting train and test data

In [45]:
df_merged.drop(columns = 'bert_input', inplace = True, axis = 1)

In [46]:
df_merged.head()

,product_id,product_locale,product_title,product_description,product_brand,esci_label,query
0,B00BO9BGYU,es,light irridiance oxigenada 20vol 1000ml l negr...,NaN,light irridiance,I,yunsey tinte amoniaco
1,3836582864,es,peter lindbergh fashion photography 40th anniv...,NaN,taschen,I,libro channel decoracion
2,B085TCJ87R,us,lenovo thinkcentre m75s small form factor sff ...,NaN,lenovo,I,ryzen 11
3,B002T3RV5S,us,delta faucet foundations single function tub s...,NaN,delta faucet,I,tub faucet without shower head
4,B082KFTF41,us,kokodi lcd writing tablet 10 inch colorful tod...,features love 1 10 inch lcd colorful screen of...,kokodi,E,educational toys 3 year old


In [47]:
df_merged.fillna("", inplace=True)

In [48]:
# Create a new column for Random Forest input
df_merged['rf_input'] = ""

# Apply functions based on the language specified in the product_locale column
for index, row in df_merged.iterrows():
    locale = row['product_locale']
    query_text = row['query']
    title_text = row['product_title']
    description_text = row['product_description']
    brand_text = row['product_brand']


    lr_input = f'{query_text} {title_text} {description_text} {brand_text}'
    df_merged.at[index, 'rf_input'] = lr_input

In [49]:
label_map = {"E": 0, "S": 1, "C": 2, "I": 3}

df_merged["esci_label"] = df_merged["esci_label"].map(label_map)

In [50]:
df_merged.head()

,product_id,product_locale,product_title,product_description,product_brand,esci_label,query,rf_input
0,B00BO9BGYU,es,light irridiance oxigenada 20vol 1000ml l negr...,,light irridiance,3,yunsey tinte amoniaco,yunsey tinte amoniaco light irridiance oxigena...
1,3836582864,es,peter lindbergh fashion photography 40th anniv...,,taschen,3,libro channel decoracion,libro channel decoracion peter lindbergh fashi...
2,B085TCJ87R,us,lenovo thinkcentre m75s small form factor sff ...,,lenovo,3,ryzen 11,ryzen 11 lenovo thinkcentre m75s small form fa...
3,B002T3RV5S,us,delta faucet foundations single function tub s...,,delta faucet,3,tub faucet without shower head,tub faucet without shower head delta faucet fo...
4,B082KFTF41,us,kokodi lcd writing tablet 10 inch colorful tod...,features love 1 10 inch lcd colorful screen of...,kokodi,0,educational toys 3 year old,educational toys 3 year old kokodi lcd writing...


In [51]:
y = df_merged['esci_label']
X = df_merged['rf_input']

In [52]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

###One-hot encoding using TF-IDF

In [53]:
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [54]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

##Creating the Random Forest model

In [55]:
# Define the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

###n_estimators Hyperparameter Tuning

In [56]:
# Define the hyperparameter grid
param_grid = {
    'n_estimators': [20, 50, 75, 100]
}

In [57]:
# Create the GridSearchCV object with micro-averaging F1-score as the scoring metric
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=5,
    scoring='f1_micro',
)

####TF-IDF

In [ ]:
# Fit the GridSearchCV object to the data
grid_search.fit(X_train_tfidf, y_train)

In [ ]:
# Print the best hyperparameter values
print("Best Hyperparameters:", grid_search.best_params_)

In [ ]:
# Get the best model
best_rf_model_tfidf = grid_search.best_estimator_

In [ ]:
# Make predictions on the test set using the best model
y_pred_tfidf = best_rf_model_tfidf.predict(X_test_tfidf)

In [ ]:
print(set(y_pred_tfidf))

In [ ]:
# Evaluate the best model using micro-averaging F1-score
micro_f1 = f1_score(y_test, y_pred_tfidf, average='micro')
print(f'Micro-Averaging F1-score of the best model: {micro_f1}')

In [ ]:
report = classification_report(y_test, y_pred_tfidf)
print('Classification Report:\n', report)

####Count Vectorizer

In [ ]:
grid_search.fit(X_train_count, y_train)

In [ ]:
# Print the best hyperparameter values
print("Best Hyperparameters:", grid_search.best_params_)

In [ ]:
# Get the best model
best_rf_model_count = grid_search.best_estimator_

In [ ]:
# Make predictions on the test set using the best model
y_pred_count = best_rf_model_count.predict(X_test_tfidf)

In [ ]:
print(set(y_pred_count))

In [ ]:
# Evaluate the best model using micro-averaging F1-score
micro_f1 = f1_score(y_test, y_pred_count, average='micro')
print(f'Micro-Averaging F1-score of the best model: {micro_f1}')

In [ ]:
report = classification_report(y_test, y_pred_count)
print('Classification Report:\n', report)